In [9]:
import tensorflow as tf

using_gpu_index = 0 # 使用的 GPU 号码
gpu_list = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_virtual_device_configuration(
        gpu_list[using_gpu_index],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024 * 4)]
    )

In [10]:
import cirq
import models.quantum_genotypes as genotypes

from models.quantum_models import generate_model_policy
from functools import reduce
env_name = "Acrobot-v1"

genotype = genotypes.ORI_TYPE_AB

n_qubits = 6 # Dimension of the state vectors in CartPole
# n_layers = 5 # Number of layers in the PQC
n_actions = 3 # Number of actions in CartPole

qubits = cirq.GridQubit.rect(1, n_qubits)
observables = [cirq.Z(qubits[0]), cirq.Z(qubits[1]), cirq.Z(qubits[2]), cirq.Z(qubits[3]), cirq.Z(qubits[4]), cirq.Z(qubits[5])] # Z_0*Z_1*Z_2*Z_3


# generate_model_policy(qubits, genotype, n_actions, beta, observables, env):
model = generate_model_policy(qubits, genotype, n_actions, 1., observables, env_name, using_H=True)

Acrobot-v1


In [11]:
from models.value_net import ValueNet

In [12]:
from IPython.core.display import HTML
from models.quantum_models import generate_circuit

circuit, params, inputs = generate_circuit(qubits, genotype, using_H=True)
display(HTML(f"<pre>{circuit}</pre>"))

In [13]:
from tensorflow.keras.optimizers import Adam
import gym
env_name = "Acrobot-v1"
env = gym.make(env_name)
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n
# hidden_dim = 64

# device = torch.device("cpu")

actor = model

critic = ValueNet()  # 价值网络
# 广义优势估计（Generalized Advantage Estimation，GAE）
# gamma 是GAE的超参数
gamma = 0.98

# lmbda 也用来计算广义优势估
lmbda = 0.95

epochs = 4  # 一条序列的数据用来训练轮数 4
eps = 0.2  # 裁剪超参数
# actor_lr = 1e-3
# critic_lr = 1e-2  # 结合 量子电路后 这个变得重要了起来
# critic_lr = 1e-1  # 结合 量子电路后 这个变得重要了起来
critic_lr = 2.5e-4 #

optimizer_in = Adam(learning_rate=0.1, amsgrad=False, epsilon=1e-5)
optimizer_var = Adam(learning_rate=0.01, amsgrad=False, epsilon=1e-5)
optimizer_out = Adam(learning_rate=0.1, amsgrad=False, epsilon=1e-5)

# Assign the model parameters to each optimizer
w_in, w_var, w_out = 1, 0, 2

# critic_optimizer = Adam(learning_rate=critic_lr)
critic_optimizer = Adam(learning_rate=critic_lr, amsgrad=False, epsilon=1e-5) 

In [14]:
from misc.utils import train

In [15]:
import numpy as np

state_bounds = np.array([1., 1., 1., 1., 12.567 , 28.274])
state_ub = state_bounds
state_lb = -1 * state_bounds

In [16]:
return_list = train(env_name, actor, critic, gamma, lmbda, eps, epochs,
          optimizer_in, optimizer_var, optimizer_out, critic_optimizer, -150.,
          critic_normalized=False, state_ub=state_ub, state_lb=state_lb, num_episodes=1000)

当前迭代次数为：0, 当前奖励为：-500.0, 平均奖励为：-500.0
当前迭代次数为：1, 当前奖励为：-500.0, 平均奖励为：-500.0
当前迭代次数为：2, 当前奖励为：-500.0, 平均奖励为：-500.0
当前迭代次数为：3, 当前奖励为：-500.0, 平均奖励为：-500.0
当前迭代次数为：4, 当前奖励为：-500.0, 平均奖励为：-500.0
当前迭代次数为：5, 当前奖励为：-500.0, 平均奖励为：-500.0
当前迭代次数为：6, 当前奖励为：-448.0, 平均奖励为：-492.57142857142856
当前迭代次数为：7, 当前奖励为：-473.0, 平均奖励为：-490.125
当前迭代次数为：8, 当前奖励为：-500.0, 平均奖励为：-491.22222222222223
当前迭代次数为：9, 当前奖励为：-500.0, 平均奖励为：-492.1
当前迭代次数为：10, 当前奖励为：-500.0, 平均奖励为：-492.1
当前迭代次数为：11, 当前奖励为：-500.0, 平均奖励为：-492.1
当前迭代次数为：12, 当前奖励为：-500.0, 平均奖励为：-492.1
当前迭代次数为：13, 当前奖励为：-500.0, 平均奖励为：-492.1
当前迭代次数为：14, 当前奖励为：-500.0, 平均奖励为：-492.1
当前迭代次数为：15, 当前奖励为：-500.0, 平均奖励为：-492.1
当前迭代次数为：16, 当前奖励为：-500.0, 平均奖励为：-497.3
当前迭代次数为：17, 当前奖励为：-500.0, 平均奖励为：-500.0
当前迭代次数为：18, 当前奖励为：-500.0, 平均奖励为：-500.0
当前迭代次数为：19, 当前奖励为：-500.0, 平均奖励为：-500.0
当前迭代次数为：20, 当前奖励为：-500.0, 平均奖励为：-500.0
当前迭代次数为：21, 当前奖励为：-500.0, 平均奖励为：-500.0
当前迭代次数为：22, 当前奖励为：-500.0, 平均奖励为：-500.0
当前迭代次数为：23, 当前奖励为：-500.0, 平均奖励为：-500.0
当前迭代次数为：24, 当前奖励为：-500.0, 平均奖励为：-500.0
当前迭代次数为

In [ ]:
# 只有量子电路规范化 Acrobot 1000次 没有收敛